<a href="https://colab.research.google.com/github/djbell01/563-DawsonBell/blob/main/Class_02-03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>